<a href="https://colab.research.google.com/github/dyllanesl/AI-EDGE-Project/blob/main/ClassiferTraining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow pillow
!pip install transformers torch mediapipe

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.15.0 requires protobuf<4.21,>=3.20.3; python_version < "3.11", but you have protobuf 4.25.3 which is incompatible.


In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("image-classification", model="google/vit-base-patch16-224")

In [ ]:
from PIL import Image
import io

# Load the uploaded image
image_path = list(uploaded.keys())[0]
image = Image.open(image_path)

# Classify the image using the pipeline
results = pipe(image)

# Print the results
for result in results:
    print(f"Label: {result['label']}, Score: {result['score']:.4f}")


Label: Windsor tie, Score: 0.2495
Label: ski mask, Score: 0.1949
Label: neck brace, Score: 0.1613
Label: wool, woolen, woollen, Score: 0.0948
Label: bulletproof vest, Score: 0.0561


In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
    js = Javascript('''
        async function takePhoto(quality) {
            const div = document.createElement('div');
            const capture = document.createElement('button');
            capture.textContent = 'Capture';
            div.appendChild(capture);

            const video = document.createElement('video');
            video.style.display = 'block';
            const stream = await navigator.mediaDevices.getUserMedia({video: true});

            document.body.appendChild(div);
            div.appendChild(video);
            video.srcObject = stream;
            await video.play();

            google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

            await new Promise((resolve) => capture.onclick = resolve);

            const canvas = document.createElement('canvas');
            canvas.width = video.videoWidth;
            canvas.height = video.videoHeight;
            canvas.getContext('2d').drawImage(video, 0, 0);
            stream.getVideoTracks()[0].stop();
            div.remove();
            return canvas.toDataURL('image/jpeg', quality);
        }
    ''')
    display(js)
    data = eval_js('takePhoto({})'.format(quality))
    binary = b64decode(data.split(',')[1])
    with open(filename, 'wb') as f:
        f.write(binary)
    return filename

# Capture photo
photo_filename = take_photo()


<IPython.core.display.Javascript object>

In [ ]:
import cv2
import mediapipe as mp
import numpy as np

MARGIN = 10 # pixels
FONT_SIZE = 1
FONT_THICKNESS = 1
HANDEDNESS_TEXT_COLOR = (88, 205, 54) # Define HANDEDNESS_TEXT_COLOR

# Function to draw landmarks on an image
def draw_landmarks_on_image(rgb_image, detection_result):
    hand_landmarks_list = detection_result.multi_hand_landmarks
    handedness_list = detection_result.multi_handedness
    annotated_image = np.copy(rgb_image)

    for idx in range(len(hand_landmarks_list)):
        hand_landmarks = hand_landmarks_list[idx]
        handedness = handedness_list[idx]

        mp.solutions.drawing_utils.draw_landmarks(
            annotated_image,
            hand_landmarks,
            mp.solutions.hands.HAND_CONNECTIONS,
            mp.solutions.drawing_styles.get_default_hand_landmarks_style(),
            mp.solutions.drawing_styles.get_default_hand_connections_style()
        )

        height, width, _ = annotated_image.shape
        x_coordinates = [landmark.x for landmark in hand_landmarks.landmark]
        y_coordinates = [landmark.y for landmark in hand_landmarks.landmark]
        text_x = int(min(x_coordinates) * width)
        text_y = int(min(y_coordinates) * height) - MARGIN

        cv2.putText(annotated_image, f"{handedness.classification[0].label}",
                    (text_x, text_y), cv2.FONT_HERSHEY_DUPLEX,
                    FONT_SIZE, HANDEDNESS_TEXT_COLOR, FONT_THICKNESS, cv2.LINE_AA)

    return annotated_image

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=True, max_num_hands=2, min_detection_confidence=0.5)

# Load the captured image
image = cv2.imread(photo_filename)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Process the image to detect hand landmarks
results = hands.process(image_rgb)

# Draw landmarks on the image
if results.multi_hand_landmarks:
    annotated_image = draw_landmarks_on_image(image_rgb, results)
    annotated_image_bgr = cv2.cvtColor(annotated_image, cv2.COLOR_RGB2BGR)
    cv2.imwrite('annotated_photo.jpg', annotated_image_bgr)
else:
    annotated_image_bgr = image
    print("No hand landmarks detected.")


No hand landmarks detected.


In [ ]:
from transformers import pipeline
from PIL import Image

# Initialize the Hugging Face image classification pipeline
pipe = pipeline("image-classification", model="google/vit-base-patch16-224")

# Load the annotated image
annotated_image_pil = Image.open('annotated_photo.jpg')

# Classify the image using the Hugging Face pipeline
classification_results = pipe(annotated_image_pil)

# Print the classification results
for result in classification_results:
    print(f"Label: {result['label']}, Score: {result['score']:.4f}")


Label: Band Aid, Score: 0.0996
Label: shower cap, Score: 0.0959
Label: paintbrush, Score: 0.0515
Label: neck brace, Score: 0.0351
Label: sweatshirt, Score: 0.0270
